In [1]:
import sqlite3

# create Database connection
conn = sqlite3.connect('group_project.db')
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS City (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
)''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS StateZip (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    code TEXT UNIQUE
)''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Housing (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date TEXT,
    price REAL,
    bedrooms INTEGER,
    bathrooms REAL,
    sqft_living INTEGER,
    sqft_lot INTEGER,
    floors REAL,
    waterfront INTEGER,
    view INTEGER,
    condition INTEGER,
    sqft_above INTEGER,
    sqft_basement INTEGER,
    yr_built INTEGER,
    yr_renovated INTEGER,
    city_id INTEGER,
    statezip_id INTEGER,
    country TEXT,
    FOREIGN KEY (city_id) REFERENCES City(id),
    FOREIGN KEY (statezip_id) REFERENCES StateZip(id)
)''')


conn.commit()#commit the changes to the database

# Function to insert and obtain foreign key
def get_or_create_fk(cursor, table, column, value):
    cursor.execute(f'SELECT id FROM {table} WHERE {column} = ?', (value,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        cursor.execute(f'INSERT INTO {table} ({column}) VALUES (?)', (value,))
        conn.commit()
        return cursor.lastrowid

# Read and insert data
with open('property.csv', 'r') as file:
    next(file)  
    for line in file:
        #print(line)
        row = line.strip().split(',')
        if len(row) < 18:  
            continue
        city_id = get_or_create_fk(cursor, 'City', 'name', row[15])
        statezip_id = get_or_create_fk(cursor, 'StateZip', 'code', row[16])
        housing_data = row[:15] + [city_id, statezip_id, row[17]]
        cursor.execute('''
        INSERT INTO Housing (
            date, price, bedrooms, bathrooms, sqft_living, sqft_lot, floors,
            waterfront, view, condition, sqft_above, sqft_basement, yr_built,
            yr_renovated, city_id, statezip_id, country
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', group_project)


conn.commit()#commit the changes to the database
conn.close()#close the connection to the database

In [9]:
pip install ydata_profiling

  Obtaining dependency information for ydata_profiling from https://files.pythonhosted.org/packages/33/85/45027914eb485482976883dcaab434eab99f1ed5cb222781e7ba46bdf3b7/ydata_profiling-4.8.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pydantic>=2 from https://files.pythonhosted.org/packages/ed/76/9a17032880ed27f2dbd490c77a3431cbc80f47ba81534131de3c2846e736/pydantic-2.7.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 1.8 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for visions[type_image_path]<0.7.7,>=0.7.5 from https://files.pythonhosted.org/packages/7c/bf/612b24e711ae25dea9af19b9304634b8949faa0b035fad47e8bcadf62f59/visions-0.7.6-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for phik<0.13,>=0.11.1 from https://files.pythonhosted.org/packages/76/7c/8cf317aca8477994318ddfcffd222e8be01b5bd9065aecf6a191da848518/phik-0.12.4-cp39-cp39-macosx_10_13_x86_64.whl.

  Obtaining dependency information for typing-extensions>=4.6.1 from https://files.pythonhosted.org/packages/01/f3/936e209267d6ef7510322191003885de524fc48d1b43269810cd589ceaf5/typing_extensions-4.11.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.5/359.5 kB 6.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.3/659.3 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 13.3 MB/s eta 0:00:00
Using cached typing_extensions-4.11.0-py3-none-any.whl (34 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 7.7 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=0613c95249a0bf0f1bc22fc19dad45230ce9bbb6697

In [2]:
#import necessary libraries
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler  # Import StandardScaler


conn = sqlite3.connect('group_project.db')#create a connection to the database

#defining a sql query to retrieve the data from the 'Housing' table and join with the 'city' and 'statezip' table
query = '''
        SELECT h.date,h.price, h.bedrooms, h.bathrooms, h.sqft_living, h.sqft_lot, h.floors,
        h.waterfront, h.view, h.condition, h.sqft_above, h.sqft_basement, h.yr_built,
        h.yr_renovated, h.country, c.name AS city, s.code AS statezip
        FROM  housing h
        JOIN city c ON h.city_id = c.id
        JOIN statezip s ON h.statezip_id = s.id;
'''
df = pd.read_sql_query(query, conn)#executing the sql query and load the results in the pandas dataframe


conn.close()#close the coonnection to the database


In [4]:
df

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,country,city,statezip


In [11]:
#import profile report
from ydata_profiling import ProfileReport
#generate a profile report for the dataframe 'df' using ProfileReport class
profile = ProfileReport(df, title="Profiling Report")
profile

ValueError: DataFrame is empty. Pleaseprovide a non-empty DataFrame.

In [12]:
#importing the matplotlib library
import matplotlib.pyplot as plt
%matplotlib inline#to ensure plots are displayed inline 
for column in train.columns:#iterating through each column in the training data
    plt.figure(figsize=(8, 6))#creating a new figure for each columns histogram
    sns.histplot(train[column], kde=True)#plot a histogram with KDE using seaborn
    plt.title(f'Distribution of {column}')#setting the title of the plot
    plt.tight_layout()#adjust layout to prevent overlapping of the subplots
    plt.show()#display the plot

NameError: name 'train' is not defined

In [148]:
from sklearn.model_selection import train_test_split#importing the necessary metrics from sklearn
#splitting the data into train and test data into 80:20
#distribution of target variable price
train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=pd.qcut(df['price'], q=10))

In [146]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Load the CSV file into a DataFrame
df = pd.read_csv('property.csv')

# Inspect the unique values and data types in the 'street' column
print(df['city'].unique())
print(df['city'].dtype)

# Handle missing values if any
df['city'].fillna('Unknown', inplace=True)

# Initialize the OneHotEncoder
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Fit and transform the 'street' column
city_encoded = onehot_encoder.fit_transform(df[['city']])

# Create a DataFrame with the encoded features
city_encoded_df = pd.DataFrame(city_encoded, columns=onehot_encoder.get_feature_names_out(['city']))

# Concatenate the encoded features with the original DataFrame
df = pd.concat([df, city_encoded_df], axis=1)

# Drop the original 'city' column
df.drop('city', axis=1, inplace=True)

# Display the DataFrame with the encoded 'street' column
print(df.head())


['Shoreline' 'Seattle' 'Kent' 'Bellevue' 'Redmond' 'Maple Valley'
 'North Bend' 'Lake Forest Park' 'Sammamish' 'Auburn' 'Des Moines'
 'Bothell' 'Federal Way' 'Kirkland' 'Issaquah' 'Woodinville'
 'Normandy Park' 'Fall City' 'Renton' 'Carnation' 'Snoqualmie' 'Duvall'
 'Burien' 'Covington' 'Inglewood-Finn Hill' 'Kenmore' 'Newcastle'
 'Mercer Island' 'Black Diamond' 'Ravensdale' 'Clyde Hill' 'Algona'
 'Skykomish' 'Tukwila' 'Vashon' 'Yarrow Point' 'SeaTac' 'Medina'
 'Enumclaw' 'Snoqualmie Pass' 'Pacific' 'Beaux Arts Village' 'Preston'
 'Milton']
object
            date      price  bedrooms  bathrooms  sqft_living  sqft_lot  \
0  5/2/2014 0:00   313000.0         3       1.50         1340      7912   
1  5/2/2014 0:00  2384000.0         5       2.50         3650      9050   
2  5/2/2014 0:00   342000.0         3       2.00         1930     11947   
3  5/2/2014 0:00   420000.0         3       2.25         2000      8030   
4  5/2/2014 0:00   550000.0         4       2.50         1940     10500

C:\Users\ideal\AppData\Local\Temp\ipykernel_37896\1943669450.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['city'].fillna('Unknown', inplace=True)
C:\Users\ideal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [202]:
train.drop('date', axis=1, inplace=True)#dropping the date column from the training data
test.drop('date',  axis=1, inplace=True)#dropping the date column from the test data

KeyError: "['date'] not found in axis"

In [170]:
train.columns


Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'sqft_above', 'sqft_basement',
       'yr_built', 'yr_renovated', 'city_Algona', 'city_Auburn',
       'city_Beaux Arts Village', 'city_Bellevue', 'city_Black Diamond',
       'city_Bothell', 'city_Burien', 'city_Carnation', 'city_Clyde Hill',
       'city_Covington', 'city_Des Moines', 'city_Duvall', 'city_Enumclaw',
       'city_Fall City', 'city_Federal Way', 'city_Inglewood-Finn Hill',
       'city_Issaquah', 'city_Kenmore', 'city_Kent', 'city_Kirkland',
       'city_Lake Forest Park', 'city_Maple Valley', 'city_Medina',
       'city_Mercer Island', 'city_Milton', 'city_Newcastle',
       'city_Normandy Park', 'city_North Bend', 'city_Pacific', 'city_Preston',
       'city_Ravensdale', 'city_Redmond', 'city_Renton', 'city_Sammamish',
       'city_SeaTac', 'city_Seattle', 'city_Shoreline', 'city_Skykomish',
       'city_Snoqualmie', 'city_Snoqualmie Pass', 'city_Tu

In [172]:
test.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'sqft_above', 'sqft_basement',
       'yr_built', 'yr_renovated', 'city_Algona', 'city_Auburn',
       'city_Beaux Arts Village', 'city_Bellevue', 'city_Black Diamond',
       'city_Bothell', 'city_Burien', 'city_Carnation', 'city_Clyde Hill',
       'city_Covington', 'city_Des Moines', 'city_Duvall', 'city_Enumclaw',
       'city_Fall City', 'city_Federal Way', 'city_Inglewood-Finn Hill',
       'city_Issaquah', 'city_Kenmore', 'city_Kent', 'city_Kirkland',
       'city_Lake Forest Park', 'city_Maple Valley', 'city_Medina',
       'city_Mercer Island', 'city_Milton', 'city_Newcastle',
       'city_Normandy Park', 'city_North Bend', 'city_Pacific', 'city_Preston',
       'city_Ravensdale', 'city_Redmond', 'city_Renton', 'city_Sammamish',
       'city_SeaTac', 'city_Seattle', 'city_Shoreline', 'city_Skykomish',
       'city_Snoqualmie', 'city_Snoqualmie Pass', 'city_Tu

In [150]:
#importing necessary metrics
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin

class DataPreprocessor(BaseEstimator, TransformerMixin):
    categorical_columns = train.select_dtypes(include=['object']).columns
    numerical_columns = train.select_dtypes(include=['int64', 'float64']).columns

    def fit(self, X, y=None): 
        # Create and fit simple imputer
        self.imputer = SimpleImputer(strategy='median')
        self.imputer.fit(X[self.numerical_columns])
        
        # Create and fit Standard Scaler 
        self.scaler = StandardScaler()
        self.scaler.fit(X[self.numerical_columns]) 
        
        # Create and fit one hot encoder
        self.onehot = OneHotEncoder(handle_unknown='ignore')
        self.onehot.fit(X[self.categorical_columns])
        
        return self 
 

    def transform(self, X): 
        # Apply simple imputer 
        imputed_cols = self.imputer.transform(X[self.numerical_columns])
        onehot_cols = self.onehot.transform(X[self.categorical_columns])
        
        # Copy the df 
        transformed_df = X.copy()
         
        # Apply transformed columns
        transformed_df[self.numerical_columns] = imputed_cols
        transformed_df[self.numerical_columns] = self.scaler.transform(transformed_df[self.numerical_columns])        
        
        # Drop existing categorical columns and replace with one hot equivalent
        transformed_df = transformed_df.drop(self.categorical_columns, axis=1) 
        transformed_df[self.onehot.get_feature_names_out()] = onehot_cols.toarray().astype(int)
        
        return transformed_df

In [200]:
preprocessor = DataPreprocessor()#create an instance of DataPreprocessor class to handle preprocessing tasks
preprocessor.fit(train)#fit the preprocessor to the training data
train_fixed = preprocessor.transform(train)#transform the training data using the fitted preprocessor

KeyError: "None of [Index(['date', 'country'], dtype='object')] are in the [columns]"

In [154]:
#importing necessary metrics
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
#creating a pipeline that includes data preprocessing using Data preprocessor
rfr = make_pipeline(DataPreprocessor(), RandomForestRegressor(n_estimators=50))

In [156]:
params = rfr.get_params()#retrieve the parameters from the random forest pipeline
params 

{'memory': None,
 'steps': [('datapreprocessor', DataPreprocessor()),
  ('randomforestregressor', RandomForestRegressor(n_estimators=50))],
 'verbose': False,
 'datapreprocessor': DataPreprocessor(),
 'randomforestregressor': RandomForestRegressor(n_estimators=50),
 'randomforestregressor__bootstrap': True,
 'randomforestregressor__ccp_alpha': 0.0,
 'randomforestregressor__criterion': 'squared_error',
 'randomforestregressor__max_depth': None,
 'randomforestregressor__max_features': 1.0,
 'randomforestregressor__max_leaf_nodes': None,
 'randomforestregressor__max_samples': None,
 'randomforestregressor__min_impurity_decrease': 0.0,
 'randomforestregressor__min_samples_leaf': 1,
 'randomforestregressor__min_samples_split': 2,
 'randomforestregressor__min_weight_fraction_leaf': 0.0,
 'randomforestregressor__n_estimators': 50,
 'randomforestregressor__n_jobs': None,
 'randomforestregressor__oob_score': False,
 'randomforestregressor__random_state': None,
 'randomforestregressor__verbose':

In [176]:
#separating the target variable price from the features in training data
y_train = train["price"]#target variable
X_train = train.drop("price",axis=1)#droppimg the price column
y_test = test['price']#target variable
X_test = test.drop('price', axis=1)#droppimg the price column

In [178]:
rfr = RandomForestRegressor()#initializing a random forest regressor model
rfr.fit(X_train, y_train)#fit the model to training data

RandomForestRegressor()

In [179]:
#importing necessary metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
y_train_hat=rfr.predict(X_train)#predict the target variable using the trained RandomForest Regressor model

In [182]:
rmse = mean_squared_error(y_train, y_train_hat)#caluculate the MSE between actual and predicted target values

In [184]:
mae = mean_absolute_error(y_train, y_train_hat)#caluculate MAE between actual and predicted target values

In [190]:
r2 = r2_score(y_train, y_train_hat)#caluculate the r2 error between actual and predicted target values

In [68]:
test.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'sqft_above', 'sqft_basement',
       'yr_built', 'yr_renovated', 'country', 'city_Algona', 'city_Auburn',
       'city_Beaux Arts Village', 'city_Bellevue', 'city_Black Diamond',
       'city_Bothell', 'city_Burien', 'city_Carnation', 'city_Clyde Hill',
       'city_Covington', 'city_Des Moines', 'city_Duvall', 'city_Enumclaw',
       'city_Fall City', 'city_Federal Way', 'city_Inglewood-Finn Hill',
       'city_Issaquah', 'city_Kenmore', 'city_Kent', 'city_Kirkland',
       'city_Lake Forest Park', 'city_Maple Valley', 'city_Medina',
       'city_Mercer Island', 'city_Milton', 'city_Newcastle',
       'city_Normandy Park', 'city_North Bend', 'city_Pacific', 'city_Preston',
       'city_Ravensdale', 'city_Redmond', 'city_Renton', 'city_Sammamish',
       'city_SeaTac', 'city_Seattle', 'city_Shoreline', 'city_Skykomish',
       'city_Snoqualmie', 'city_Snoqualmie Pass

In [164]:
train.drop('country', axis=1, inplace=True)#dropping the country column from the training data
test.drop('country',  axis=1, inplace=True)#dropping the country column from the test data

In [212]:
y_train.columns

AttributeError: 'Series' object has no attribute 'columns'

In [216]:
import mlflow
from mlflow.models import infer_signature

# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("Housing Price Prediction")
experiment_name = "RANDOM_FOREST_REGRESSI0N"
try:
    mlflow.set_experiment(experiment_name)
except MlflowException as e:
    if "RESOURCE_DOES_NOT_EXIST" in str(e):
        mlflow.create_experiment(experiment_name)
        mlflow.set_experiment(experiment_name)
    else:
        raise e
# Start an MLflow run

model = RandomForestRegressor()
model.fit(X_train, y_train)

with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log metrics
    mlflow.log_metric("root_mean_squared_error", rmse)
    mlflow.log_metric("mean_absolute_error", mae)
    mlflow.log_metric("r2_score", r2)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Model Info", "RandomForestRegressor for housing data")

    # Infer the model signature
    signature = infer_signature(X_train, model.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="housing_model",
        signature=signature,
        registered_model_name="rfr_housing_model",
    )

C:\Users\ideal\anaconda3\Lib\site-packages\mlflow\types\utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
C:\Users\ideal\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'rfr_housing_model'.
2024/05/15 22:59:

In [230]:
pip install xgboost

   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 217.9 kB/s eta 0:07:38
   ---------------------------------------- 0.1/99.8 MB 469.7 kB/s eta 0:03:33
   ---------------------------------------- 0.3/99.8 MB 1.9 MB/s eta 0:00:53
   ---------------------------------------- 1.2/99.8 MB 5.2 MB/s eta 0:00:19
   - -------------------------------------- 2.5/99.8 MB 10.0 MB/s eta 0:00:10
   - -------------------------------------- 4.1/99.8 MB 13.8 MB/s eta 0:00:07
   -- ------------------------------------- 6.0/99.8 MB 17.5 MB/s eta 0:00:06
   --- ------------------------------------ 8.1/99.8 MB 20.0 MB/s eta 0:00:05
   --- ------------------------------------ 10.0/99.8 MB 21.9 MB/s eta 0:00:05
   ---- ----------------------------------- 11.8/99.8 MB 38.5 MB/s eta 0:00:03
   ----- ---------------------------------- 14.2/99.8 MB 40.9 MB/s eta 0:00:03

In [232]:
import mlflow
from mlflow.models import infer_signature
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
experiment_name = "Housing Price Prediction"
try:
    mlflow.set_experiment(experiment_name)
except mlflow.exceptions.MlflowException as e:
    if "RESOURCE_DOES_NOT_EXIST" in str(e):
        mlflow.create_experiment(experiment_name)
        mlflow.set_experiment(experiment_name)
    else:
        raise e

# Define the model
model = XGBRegressor()

# Fit the model
model.fit(X_train, y_train)

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(model.get_params())

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate and log metrics
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mlflow.log_metric("root_mean_squared_error", rmse)
    mlflow.log_metric("mean_absolute_error", mae)
    mlflow.log_metric("r2_score", r2)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Model Info", "XGBRegressor for housing data")

    # Infer the model signature
    signature = infer_signature(X_train, model.predict(X_train))

    # Log the model
    model_info = mlflow.xgboost.log_model(
        xgb_model=model,
        artifact_path="housing_model",
        signature=signature,
        registered_model_name="xgb_housing_model",
    )


C:\Users\ideal\anaconda3\Lib\site-packages\mlflow\types\utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
C:\Users\ideal\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:26:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving i

In [238]:
import mlflow
from mlflow.models import infer_signature
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
experiment_name = "Housing Price Prediction"
try:
    mlflow.set_experiment(experiment_name)
except mlflow.exceptions.MlflowException as e:
    if "RESOURCE_DOES_NOT_EXIST" in str(e):
        mlflow.create_experiment(experiment_name)
        mlflow.set_experiment(experiment_name)
    else:
        raise e

# Define the model using BaggingRegressor
model = BaggingRegressor()

# Fit the model
model.fit(X_train, y_train)

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(model.get_params())

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate and log metrics
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mlflow.log_metric("root_mean_squared_error", rmse)
    mlflow.log_metric("mean_absolute_error", mae)
    mlflow.log_metric("r2_score", r2)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Model Info", "BaggingRegressor for housing data")

    # Infer the model signature
    signature = infer_signature(X_train, model.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="housing_model",
        signature=signature,
        registered_model_name="bagging_housing_model",
    )


C:\Users\ideal\anaconda3\Lib\site-packages\mlflow\types\utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Successfully registered model 'bagging_housing_model'.
2024/05/15 23:40:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: bagging_housing_model, version 1
Created vers

In [240]:
train.drop('view', axis=1, inplace=True)#dropping the view column from the training data
test.drop('view',  axis=1, inplace=True)#dropping the view column from the test data
train.drop('waterfront', axis=1, inplace=True)#dropping the waterfront column from the training data
test.drop('waterfront',  axis=1, inplace=True)#dropping the waterfront column from the test data

In [242]:
import mlflow
from mlflow.models import infer_signature
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
experiment_name = "Housing Price Prediction"
try:
    mlflow.set_experiment(experiment_name)
except mlflow.exceptions.MlflowException as e:
    if "RESOURCE_DOES_NOT_EXIST" in str(e):
        mlflow.create_experiment(experiment_name)
        mlflow.set_experiment(experiment_name)
    else:
        raise e

# Define the model using GradientBoostingRegressor
model = GradientBoostingRegressor()

# Fit the model
model.fit(X_train, y_train)

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(model.get_params())

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate and log metrics
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mlflow.log_metric("root_mean_squared_error", rmse)
    mlflow.log_metric("mean_absolute_error", mae)
    mlflow.log_metric("r2_score", r2)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Model Info", "GradientBoostingRegressor for housing data")

    # Infer the model signature
    signature = infer_signature(X_train, model.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="housing_model",
        signature=signature,
        registered_model_name="gbr_housing_model",
    )


C:\Users\ideal\anaconda3\Lib\site-packages\mlflow\types\utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Successfully registered model 'gbr_housing_model'.
2024/05/15 23:47:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: gbr_housing_model, version 1
Created version '1' 